# Covid-19: From model prediction to model predictive control

## Calibration of the age-stratified national model (google)

*Original code by Ryan S. McGee. Modified by T.W. Alleman in consultation with the BIOMATH research unit headed by prof. Ingmar Nopens.*

Copyright (c) 2020 by T.W. Alleman, BIOMATH, Ghent University. All Rights Reserved.

This notebook accompanies our preprint: "*A deterministic, age-stratified, extended SEIRD model for assessing the effect of non-pharmaceutical interventions on SARS-CoV-2 spread in Belgium*"(https://doi.org/10.1101/2020.07.17.20156034)

### Load required packages

In [10]:
import random
import os
import numpy as np
import json
import corner
import random

import pandas as pd
import datetime
import scipy
import matplotlib.dates as mdates
import matplotlib
import math
import xarray as xr
import emcee
import matplotlib.pyplot as plt
import datetime

from covid19model.optimization import objective_fcns,pso
from covid19model.models import models
from covid19model.models.utils import draw_sample_COVID19_SEIRD_google
from covid19model.models.time_dependant_parameter_fncs import google_lockdown_no_prev
from covid19model.data import google
from covid19model.data import sciensano
from covid19model.data import model_parameters
from covid19model.visualization.output import population_status, infected, _apply_tick_locator 
from covid19model.visualization.optimization import plot_fit, traceplot

# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

### Load interaction matrices

In [2]:
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(dataset='willem_2012')
levels = initN.size

In [3]:
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(dataset='willem_2012',intensity='all')
initN, Nc_home_15, Nc_work_15, Nc_schools_15, Nc_transport_15, Nc_leisure_15, Nc_others_15, Nc_total_15 = model_parameters.get_interaction_matrices(dataset='willem_2012',intensity='more_15_min')
initN, Nc_home_1hr, Nc_work_1hr, Nc_schools_1hr, Nc_transport_1hr, Nc_leisure_1hr, Nc_others_1hr, Nc_total_1hr = model_parameters.get_interaction_matrices(dataset='willem_2012',intensity='more_one_hour')

In [4]:
Nc_all = {'total': Nc_total, 'home':Nc_home, 'work': Nc_work, 'schools': Nc_schools, 'transport': Nc_transport, 'leisure': Nc_leisure, 'others': Nc_others}
Nc_15min = {'total': Nc_total_15, 'home': Nc_home_15, 'work': Nc_work_15, 'schools': Nc_schools_15, 'transport': Nc_transport_15, 'leisure': Nc_leisure_15, 'others': Nc_others_15}
Nc_1hr = {'total': Nc_total_1hr, 'home': Nc_home_1hr, 'work': Nc_work_1hr, 'schools': Nc_schools_1hr, 'transport': Nc_transport_1hr, 'leisure': Nc_leisure_1hr, 'others': Nc_others_1hr}

### Scrape high-level Sciensano data

In [5]:
df_sciensano = sciensano.get_sciensano_COVID19_data(update=False)
df_sciensano.tail()

,H_tot,ICU_tot,H_in,H_out,H_tot_cumsum,D_tot,D_25_44,D_45_64,D_65_74,D_75_84,D_85+
DATE,,,,,,,,,,,
2020-11-08,6955,1470,403,295,5669,204,1.0,11.0,29,55,107
2020-11-09,7221,1474,434,145,5958,185,0.0,13.0,25,58,89
2020-11-10,7058,1470,609,708,5859,163,1.0,15.0,21,47,78
2020-11-11,6879,1463,544,626,5777,121,1.0,10.0,15,46,49
2020-11-12,7010,1452,399,242,5934,19,0.0,2.0,3,5,9


### Scrape Google Community Mobility Reports

In [6]:
df_google = google.get_google_mobility_data(update=False, plot=False)

### Load time-dependant parameters

In [7]:
from covid19model.models.time_dependant_parameter_fncs import google_lockdown

def switch_beta(t,param,samples_dict):
    if t < pd.to_datetime('2020-05-04'):
        return np.random.choice(samples_dict['beta'],1,replace=False)
    elif pd.to_datetime('2020-05-04') < t <= pd.to_datetime('2020-09-01'):
        return np.random.choice(samples_dict['beta_summer'],1,replace=False)
    else:
        return np.random.choice(samples_dict['beta'],1,replace=False)

# Wave 2: September 2020 - present

## Recalibrate ramp as final calibration step

In [8]:
# Load samples dictionary of the first wave
with open('../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/google/BE_2020-11-10_WAVE1_GOOGLE.json', 'r') as fp:
    samples_dict = json.load(fp)

with open('../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/google/initial_states_2020-09-01.json', 'r') as fp:
    initial_states = json.load(fp)    

In [16]:
# Start of data collection
start_data = '2020-09-01'
# Start data of recalibration ramp
start_calibration = '2020-09-01'
# Last datapoint used to recalibrate the ramp
end_calibration = '2020-11-12'
# Path where figures should be stored
fig_path = '../../results/calibrations/COVID19_SEIRD/national/'
# Path where MCMC samples should be saved
samples_path = '../../data/interim/model_parameters/COVID19_SEIRD/calibrations/national/'
# Spatial unit: Belgium
spatial_unit = 'BE_no_prev'

In [17]:
# Load the model parameters using `get_COVID19_SEIRD_parameters()`.
params = model_parameters.get_COVID19_SEIRD_parameters()

params.update({'df_google': df_google,
              'Nc_all' : Nc_all,
              'Nc_15min' : Nc_15min,
              'Nc_1hr' : Nc_1hr,
               'l' : 5,
               'tau' : 5,
              })

# Initialize the model
model = models.COVID19_SEIRD(initial_states, params, time_dependent_parameters={'Nc': google_lockdown_no_prev})

In [ ]:
warmup=0
maxiter = 100
popsize = 100
steps_mcmc = 3000
discard = 1000

# define dataset
data=[df_sciensano['H_in'][start_calibration:end_calibration]]
states = [["H_in"]]

####################################################
####### CALIBRATING BETA AND COMPLIANCE RAMP #######
####################################################

print('------------------------------------')
print('CALIBRATING BETA AND COMPLIANCE RAMP')
print('------------------------------------\n')
print('Using data from '+start_calibration+' until '+end_calibration+'\n')
print('1) Particle swarm optimization\n')

# set PSO optimisation settings
parNames = ['sigma_data','beta','l','tau']
bounds=((1,2000),(0.010,0.060),(0.1,20),(0.1,20))
# run PSO optimisation
theta = pso.fit_pso(model,data,parNames,states,bounds,maxiter=maxiter,popsize=popsize,start_date=start_calibration,warmup=warmup)

# run MCMC sampler
print('\n2) Markov-Chain Monte-Carlo sampling\n')
parNames_mcmc = parNames
bounds_mcmc=((1,2000),(0.020,0.060),(0.001,20),(0.001,20))

pos = theta + [1, 1e-3, 1e-3, 1e-3]* np.random.randn(10, 4)
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(nwalkers, ndim, objective_fcns.log_probability,
                    args=(model, bounds_mcmc, data, states, parNames_mcmc, None, start_calibration, warmup))
sampler.run_mcmc(pos, steps_mcmc, progress=True)

try:
    sampler.get_autocorr_time()
except:
    print('Warning: The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s).\nUse this estimate with caution and run a longer chain!')

from covid19model.optimization.run_optimization import checkplots
checkplots(sampler.get_chain(discard=discard,flat=False), sampler.get_chain(discard=discard,flat=True), fig_path, spatial_unit, 
            figname='BETA_RAMP_GOOGLE_WAVE2', labels=['$\sigma_{data}$','$\\beta$','l','$\\tau$'])

#############################################
####### Output to dictionary ################
#############################################

samples_dict_wave2 = {
    'beta': sampler.get_chain(discard=discard,flat=True)[:,1].tolist(),
    'l' : sampler.get_chain(discard=discard,flat=True)[:,2].tolist(),
    'tau' : sampler.get_chain(discard=discard,flat=True)[:,3].tolist(),
}

with open(samples_path+str(spatial_unit)+'_'+str(datetime.date.today())+'_WAVE2_GOOGLE.json', 'w') as fp:
    json.dump(samples_dict_wave2, fp)

------------------------------------
CALIBRATING BETA AND COMPLIANCE RAMP
------------------------------------

Using data from 2020-09-01 until 2020-11-12

1) Particle swarm optimization

No constraints given.
Best after iteration 1: [1.24747930e+02 4.22775603e-02 6.39815437e+00 7.11236861e+00] 373.8517369306902
Best after iteration 2: [1.24747930e+02 4.22775603e-02 6.39815437e+00 7.11236861e+00] 373.8517369306902
Best after iteration 3: [1.24747930e+02 4.22775603e-02 6.39815437e+00 7.11236861e+00] 373.8517369306902
New best for swarm at iteration 4: [8.38811273e+01 4.28216264e-02 1.39968299e+01 1.19169771e+00] 362.78646235045903
Best after iteration 4: [8.38811273e+01 4.28216264e-02 1.39968299e+01 1.19169771e+00] 362.78646235045903
Best after iteration 5: [8.38811273e+01 4.28216264e-02 1.39968299e+01 1.19169771e+00] 362.78646235045903
Best after iteration 6: [8.38811273e+01 4.28216264e-02 1.39968299e+01 1.19169771e+00] 362.78646235045903
New best for swarm at iteration 7: [9.29882209

Best after iteration 55: [6.12278105e+01 4.40566485e-02 3.90816081e+00 1.14478062e-01] 337.13501852093333
New best for swarm at iteration 56: [6.14438469e+01 4.40623397e-02 3.90697603e+00 1.15995312e-01] 337.1307257177758
Best after iteration 56: [6.14438469e+01 4.40623397e-02 3.90697603e+00 1.15995312e-01] 337.1307257177758
Best after iteration 57: [6.14438469e+01 4.40623397e-02 3.90697603e+00 1.15995312e-01] 337.1307257177758
Best after iteration 58: [6.14438469e+01 4.40623397e-02 3.90697603e+00 1.15995312e-01] 337.1307257177758
Best after iteration 59: [6.14438469e+01 4.40623397e-02 3.90697603e+00 1.15995312e-01] 337.1307257177758
Best after iteration 60: [6.14438469e+01 4.40623397e-02 3.90697603e+00 1.15995312e-01] 337.1307257177758
Best after iteration 61: [6.14438469e+01 4.40623397e-02 3.90697603e+00 1.15995312e-01] 337.1307257177758
Best after iteration 62: [6.14438469e+01 4.40623397e-02 3.90697603e+00 1.15995312e-01] 337.1307257177758
Best after iteration 63: [6.14438469e+01 4.

 12%|█▏        | 362/3000 [28:32<3:31:43,  4.82s/it]

## Visualize fit

In [ ]:
end_sim = '2021-01-01'

fig,ax=plt.subplots(figsize=(10,4))
for i in range(200):
    # Sampling
    model.parameters['beta'] = np.random.choice(sampler.get_chain(discard=discard,flat=True)[:,1].tolist(),1,replace=False)[0]
    idx,model.parameters['l'] = random.choice(list(enumerate(sampler.get_chain(discard=discard,flat=True)[:,2].tolist())))
    model.parameters['tau'] = sampler.get_chain(discard=discard,flat=True)[:,3].tolist()[idx]    
    model.parameters['prevention'] = sampler.get_chain(discard=discard,flat=True)[:,4].tolist()[idx]    
    # Simulate
    y_model = model.sim(end_sim,start_date=start_calibration,warmup=0)
    # Plot
    ax.plot(y_model['time'],y_model["H_in"].sum(dim="Nc"),color='blue',alpha=0.01)

ax.scatter(df_sciensano[start_calibration:end_calibration].index,df_sciensano['H_in'][start_calibration:end_calibration],color='black',alpha=0.6,linestyle='None',facecolors='none')
ax = _apply_tick_locator(ax)
ax.set_xlim('2020-09-01',end_sim)
plt.savefig(fig_path+'others/BETA_RAMP_FIT_WAVE2_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')

## Make a prediction under current contact behaviour

In [ ]:
end_sim = '2021-02-01'
n=200
percentile = 0.95

y_model = model.sim(end_sim,start_date=start_calibration,warmup=0,N=n,draw_fcn=draw_sample_COVID19_SEIRD_google,samples=samples_dict_wave2)

fig,ax = plt.subplots(figsize=(10,4))
ax.fill_between(pd.to_datetime(y_model['time'].values),y_model["H_tot"].quantile(1-percentile,dim="draws").sum(dim="Nc"), y_model["H_tot"].quantile(percentile,dim="draws").sum(dim="Nc"),alpha=0.20, color = 'blue')
ax.fill_between(pd.to_datetime(y_model['time'].values),y_model["H_tot"].quantile(1-0.68,dim="draws").sum(dim="Nc"), y_model["H_tot"].quantile(0.68,dim="draws").sum(dim="Nc"),alpha=0.35, color = 'blue')
ax.plot(y_model['time'],y_model["H_tot"].mean(dim="draws").sum(dim="Nc"),'--', color='blue')
ax.scatter(df_sciensano[start_calibration:end_calibration].index,df_sciensano['H_tot'][start_calibration:end_calibration],color='black',alpha=0.4,linestyle='None',facecolors='none')
ax.set_ylabel('Total patients in Belgian hospitals')
ax = _apply_tick_locator(ax)
plt.savefig(fig_path+'others/LOCKDOWN_WAVE2_GOOGLE.pdf', dpi=400, bbox_inches='tight',orientation='portrait', papertype='a4')